## Fake News Classifier Using LSTM - Sentiment Analysis

Dataset: https://www.kaggle.com/c/fake-news/data#

In [63]:
import pandas as pd

In [64]:
df=pd.read_csv('/content/train.csv')

In [65]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [66]:
###Drop Nan Values
df=df.dropna()


In [67]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [68]:
## Get the Dependent features
y=df['label']

In [69]:
X.shape

(18285, 4)

In [70]:
y.shape

(18285,)

In [71]:
import tensorflow as tf

In [72]:
tf.__version__

'2.17.0'

In [73]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [74]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [75]:
messages=X.copy()

In [76]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [77]:
messages.reset_index(inplace=True)

In [78]:
import nltk
import re
from nltk.corpus import stopwords

In [79]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [80]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [81]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [82]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[1215, 4594, 4356, 1525, 4737, 445, 4079, 3973, 2730, 4303],
 [4502, 2138, 611, 65, 2524, 357, 3169],
 [4887, 1395, 2703, 1806],
 [4685, 1763, 59, 308, 2063, 4163],
 [4031, 2524, 3663, 4179, 1524, 2712, 2524, 2625, 171, 1822],
 [2136,
  1895,
  2789,
  1855,
  2595,
  2128,
  2290,
  3988,
  3522,
  1550,
  1303,
  45,
  122,
  3086,
  3169],
 [4129, 2087, 2180, 321, 1729, 1450, 1268, 2806, 3008, 1944, 4676],
 [4519, 2375, 3549, 975, 3342, 2760, 2128, 4478, 3008, 1944, 4676],
 [2407, 442, 3313, 2766, 4735, 3625, 2603, 1902, 2128, 3414],
 [3736, 809, 1341, 1746, 1759, 41, 447, 1930],
 [2485, 2051, 4763, 4058, 2062, 3419, 3761, 3637, 4935, 3686, 4070],
 [308, 1684, 4737, 3625, 2128, 3342],
 [1233, 3620, 77, 4965, 3963, 4061, 1374, 4651, 1972],
 [2153, 3495, 4933, 193, 1543, 4899, 4293, 3008, 1944, 4676],
 [1857, 3154, 2093, 2046, 4822, 3008, 1944, 4676],
 [3332, 3231, 250, 1424, 3486, 2726, 765, 2134, 478, 801],
 [3185, 1573, 2138],
 [742, 181, 4340, 4999, 2128, 519, 2210, 3169],
 [2401

### Embedding Representation

In [83]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3973 2730 4303]
 [   0    0    0 ... 2524  357 3169]
 [   0    0    0 ... 1395 2703 1806]
 ...
 [   0    0    0 ... 3008 1944 4676]
 [   0    0    0 ...  614 3861 4729]
 [   0    0    0 ...  735 4257 3567]]


In [84]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1215,
       4594, 4356, 1525, 4737,  445, 4079, 3973, 2730, 4303], dtype=int32)

In [85]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [87]:
len(embedded_docs),y.shape

(18285, (18285,))

In [88]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [89]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [90]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [91]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - accuracy: 0.8000 - loss: 0.4303 - val_accuracy: 0.9107 - val_loss: 0.2096
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.9360 - loss: 0.1574 - val_accuracy: 0.9142 - val_loss: 0.2045
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 50ms/step - accuracy: 0.9611 - loss: 0.1026 - val_accuracy: 0.9127 - val_loss: 0.2397
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.9766 - loss: 0.0686 - val_accuracy: 0.9084 - val_loss: 0.2830
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 9s 47ms/step - accuracy: 0.9833 - loss: 0.0562 - val_accuracy: 0.9090 - val_loss: 0.2795
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 44ms/step - accuracy: 0.9866 - loss: 0.0458 - val_accuracy: 0.9074 - val_loss: 0.3480
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.9916 - loss: 0.0318 - val_accuracy: 0.9090 - val_loss: 0.3916
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 51ms/step - accuracy: 0.9936 - loss: 0.0227 - val_

### Adding Dropout

In [92]:
# from tensorflow.keras.layers import Dropout
# ## Creating model
# embedding_vector_features=40
# model=Sequential()
# model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# model.add(Dropout(0.3))
# model.add(LSTM(100))
# model.add(Dropout(0.3))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [94]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step


In [95]:
from sklearn.metrics import confusion_matrix

In [96]:
confusion_matrix(y_test,y_pred)

array([[3078,  341],
       [ 230, 2386]])

In [97]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9053852526926264